In [1]:
import xarray as xr
import numpy as np

from datetime import datetime
from glob import glob

from netCDF4 import default_fillvals  # pylint: disable=no-name-in-module

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_decimal_year(dt: datetime):
    year_start = datetime(dt.year, 1, 1)
    year_end = year_start.replace(year=dt.year+1)
    return dt.year + ((dt - year_start).total_seconds() /  # seconds so far
        float((year_end - year_start).total_seconds()))  # seconds in year

def interp(ds:xr.Dataset) -> xr.Dataset:
    new_lats = np.arange(-89.875,90.125,0.25)
    new_lons = np.arange(-9.825,369.825, 0.25)
    interp_ds = ds.interp(longitude=new_lons, latitude=new_lats)
    return interp_ds


def smoothing(ds):
    interp_ds = interp(ds)
    dsr = interp_ds.rolling({'longitude':38, 'latitude':16}, min_periods=1, center=True).mean()
    dsr = dsr.sel(longitude=slice(0,360))
    
    # Do boxcar averaging
    hr_mask_ds = xr.open_dataset('/Users/marlis/Developer/SLI/Sea-Level-Indicators/ref_files/HR_GRID_MASK_latlon.nc')
    hr_mask_ds.coords['longitude'] = hr_mask_ds.coords['longitude'] % 360
    hr_mask_ds = hr_mask_ds.sortby(hr_mask_ds.longitude)

    dsr.SSHA.values = np.where(hr_mask_ds.maskC.values == 0, np.nan, dsr.SSHA.values)

    filtered_ds = dsr.where(dsr.counts > 475, np.nan)
    filtered_ds.SSHA.values = np.where(hr_mask_ds.maskC.values == 0, np.nan, filtered_ds.SSHA.values)

    dsr_subset = filtered_ds.sel(latitude=slice(-82,82))

    return dsr_subset

In [3]:
simple_grid_paths = glob(f'/Users/marlis/Developer/SLI/sli_output/gridded_cycles/*.nc')
simple_grid_paths.sort()

seas_ds = xr.open_dataset('../../ref_files/trnd_seas_simple_grid.nc')
seas_ds.coords['Longitude'] = (seas_ds.coords['Longitude']) % 360
seas_ds = seas_ds.sortby(seas_ds.Longitude)

front_seas_ds = seas_ds.isel(Month_grid=0)
back_seas_ds = seas_ds.isel(Month_grid=-1)
front_seas_ds = front_seas_ds.assign_coords({'Month_grid': front_seas_ds.Month_grid.values + (12/12)})
back_seas_ds = back_seas_ds.assign_coords({'Month_grid': back_seas_ds.Month_grid.values - (12/12)})
padded_seas_ds = xr.concat([back_seas_ds, seas_ds, front_seas_ds], dim='Month_grid')

hr_mask_ds = xr.open_dataset('/Users/marlis/Developer/SLI/Sea-Level-Indicators/ref_files/HR_GRID_MASK_latlon.nc')
hr_mask_ds.coords['longitude'] = hr_mask_ds.coords['longitude'] % 360
hr_mask_ds = hr_mask_ds.sortby(hr_mask_ds.longitude)

for f in simple_grid_paths:
    file_name = f.split('/')[-1]
    if file_name >= 'ssha_global_half_deg_20231101.nc':
        ds = xr.open_dataset(f)
        ds.coords['longitude'] = (ds.coords['longitude']) % 360
        ds = ds.sortby(ds.longitude)
        ds = ds.where(ds.counts > 475, np.nan)

        lats = ds.latitude.values
        lons = ds.longitude.values
        data = ds.SSHA.values * 1000
        counts = ds.counts.values
        date = datetime.utcfromtimestamp(ds.time.values.tolist()/1e9)
        print(date)
        date_str = datetime.strftime(date, '%b %d %Y')

        outdir = '/Users/marlis/Developer/SLI/sli_output/ENSO_grids'
        filename = f'ENSO_{datetime.strftime(date, "%Y%m%d")}.nc'

        decimal_year = get_decimal_year(date)
        yr_fraction = decimal_year - date.year
        cycle_ds = padded_seas_ds.interp({'Month_grid': yr_fraction})

        removed_cycle_data = data - (cycle_ds.Seasonal_SSH.values * 10)
        trend = (decimal_year * seas_ds.SSH_Slope * 10) + (seas_ds.SSH_Offset * 10)
        removed_cycle_trend_data = removed_cycle_data - trend
        ds.SSHA.values = removed_cycle_trend_data

        front = ds.sel(longitude=slice(0,10))
        back = ds.sel(longitude=slice(350,360))
        front = front.assign_coords({'longitude': front.longitude.values + 360})
        back = back.assign_coords({'longitude': back.longitude.values - 360})
        padded_ds = xr.merge([back, ds, front])

        smooth_ds = smoothing(padded_ds)
        filtered_ds = smooth_ds.drop_vars(['counts', 'mask'])

        filtered_ds.SSHA.attrs = ds.SSHA.attrs
        filtered_ds.SSHA.attrs['units'] = 'mm'
        filtered_ds.SSHA.attrs['valid_min'] = np.nanmin(filtered_ds.SSHA.values)
        filtered_ds.SSHA.attrs['valid_max'] = np.nanmax(filtered_ds.SSHA.values)
        filtered_ds.SSHA.attrs['summary'] = 'Data gridded to 0.25 degree grid with boxcar smoothing applied'

        filtered_ds.time.attrs = {'long_name': 'time', 'standard_name': 'time'}

        filtered_ds.latitude.attrs = {'long_name': 'latitude', 'standard_name': 'latitude'}
        filtered_ds.longitude.attrs = {'long_name': 'longitude', 'standard_name': 'longitude'}

        outdir = '/Users/marlis/Developer/SLI/sli_output/ENSO_grids'
        filename = f'ENSO_{datetime.strftime(date, "%Y%m%d")}.nc'

        var_encoding = {'zlib': True,
                        'complevel': 5,
                        'dtype': 'float32',
                        'shuffle': True,
                        '_FillValue': default_fillvals['f8']}
        encoding = {var: var_encoding for var in filtered_ds.data_vars}
        encoding['time'] = {'units' : 'days since 1985-01-01'}
        filtered_ds.to_netcdf(f'{outdir}/{filename}', encoding=encoding)

2023-11-06 00:00:00
2023-11-13 00:00:00
2023-11-20 00:00:00
2023-11-27 00:00:00


In [6]:
# MONTHLY AVERAGING
outdir = '/Users/marlis/Developer/SLI/sli_output/monthly_ENSO_grids'
dates = [str(d).replace('-', '') for d in np.arange('2023-05-01', 'now', 1, dtype='datetime64[M]')]
for d in dates:
    files = glob(f'/Users/marlis/Developer/SLI/sli_output/ENSO_grids/*{d}*.nc')
    files.sort()

    if not files or len(files) < 4:
        continue

    all_data = [xr.open_dataset(f) for f in files]
    ds = xr.concat(all_data, dim='time').mean('time', keep_attrs=True)
    ds.SSHA.attrs['valid_min'] = np.nanmin(ds.SSHA.values)
    ds.SSHA.attrs['valid_max'] = np.nanmax(ds.SSHA.values)
    ds.SSHA.attrs['summary'] = f'Monthly average of {ds.SSHA.attrs["summary"].lower()}'
    ds = ds.assign_coords({'time': [datetime(int(d[:4]), int(d[4:6]), 1)]})
    var_encoding = {'zlib': True,
                    'complevel': 5,
                    'dtype': 'float32',
                    'shuffle': True,
                    '_FillValue': default_fillvals['f8']}
    encoding = {var: var_encoding for var in filtered_ds.data_vars}
    encoding['time'] = {'units' : 'days since 1985-01-01'}
    filename = f'monthly_ENSO_{d}.nc'
    ds.to_netcdf(f'{outdir}/{filename}', encoding = encoding)